In [ ]:
# 1. 콘텐츠 기반 추천 (Content-Based Filtering):

# 사용자의 과거 선호도를 바탕으로 유사한 콘텐츠를 추천합니다.
# 사용자의 프로필, 취향, 검색 기록 등을 분석하여 추천합니다.

# 2. 협업 필터링 (Collaborative Filtering):

# 다른 사용자들의 선호도 데이터를 활용하여 추천합니다.
# 유사한 취향을 가진 사용자들의 선호도를 기반으로 추천합니다.

# 3. 하이브리드 추천 (Hybrid Recommendation):

# 콘텐츠 기반 추천과 협업 필터링을 결합한 방식입니다.
# 두 가지 방식의 장점을 활용하여 더 정확한 추천을 제공합니다.

# 4. 인구통계학적 추천 (Demographic-Based Recommendation):

# 사용자의 인구통계학적 특성(나이, 성별, 지역 등)을 활용하여 추천합니다.
# 유사한 특성을 가진 사용자들의 선호도를 기반으로 추천합니다.

# 5. 지식 기반 추천 (Knowledge-Based Recommendation):

# 사용자의 요구사항과 제품의 특성을 매칭하여 추천합니다.
# 도메인 지식을 활용하여 추천합니다.

# 6. 순차 패턴 기반 추천 (Sequential Pattern-Based Recommendation):

# 사용자의 이전 행동 패턴을 분석하여 추천합니다.
# 사용자의 최근 행동을 기반으로 다음 행동을 예측하여 추천합니다.

In [ ]:
# 행렬분해

In [ ]:
# 콘텐츠 기반 추천
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def content_based_recommender(user_profile, item_features, top_n=5):
    # 아이템 특징과 사용자 프로필 간의 코사인 유사도 계산
    similarities = cosine_similarity(item_features, user_profile)
    
    # 유사도 기반으로 상위 N개의 아이템 추천
    top_indices = np.argsort(similarities)[::-1][:top_n]
    recommended_items = [item_features[i] for i in top_indices]
    
    return recommended_items

In [ ]:
# 협업 필터링
import numpy as np
from scipy.spatial.distance import cosine

def collaborative_filtering_recommender(user_ratings, top_n=5):
    # 사용자-아이템 평점 행렬 계산
    user_item_matrix = np.array(user_ratings)
    
    # 사용자 간 코사인 유사도 계산
    user_similarities = 1 - user_item_matrix.T.dot(user_item_matrix) / (np.linalg.norm(user_item_matrix, axis=1, keepdims=True) * np.linalg.norm(user_item_matrix, axis=1, keepdims=True).T)
    
    # 현재 사용자와 가장 유사한 상위 N명의 사용자 선택
    top_user_indices = np.argsort(user_similarities[0])[::-1][1:top_n+1]
    
    # 유사한 사용자들의 평점을 기반으로 아이템 추천
    recommended_items = np.argsort(user_item_matrix[:, top_user_indices].mean(axis=1))[::-1][:top_n]
    
    return recommended_items

In [ ]:
# 하이브리드
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def hybrid_recommender(user_profile, item_features, user_ratings, top_n=5):
    # 콘텐츠 기반 추천
    content_based_scores = cosine_similarity(item_features, user_profile)
    
    # 협업 필터링 기반 추천
    user_item_matrix = np.array(user_ratings)
    user_similarities = 1 - user_item_matrix.T.dot(user_item_matrix) / (np.linalg.norm(user_item_matrix, axis=1, keepdims=True) * np.linalg.norm(user_item_matrix, axis=1, keepdims=True).T)
    collaborative_scores = user_item_matrix[:, np.argsort(user_similarities[0])[::-1][1:top_n+1]].mean(axis=1)
    
    # 콘텐츠 기반 점수와 협업 필터링 점수를 합산하여 최종 추천 점수 계산
    hybrid_scores = content_based_scores + collaborative_scores
    
    # 상위 N개의 추천 아이템 선택
    top_indices = np.argsort(hybrid_scores)[::-1][:top_n]
    recommended_items = [item_features[i] for i in top_indices]
    
    return recommended_items


In [ ]:
# 지식 기반 추천
import networkx as nx
from operator import itemgetter

def knowledge_based_recommender(user_profile, item_knowledge_graph, top_n=5):
    # 지식 그래프 생성
    G = nx.Graph()
    for item, attributes in item_knowledge_graph.items():
        G.add_node(item, **attributes)
        for related_item, weight in attributes['related_items'].items():
            G.add_edge(item, related_item, weight=weight)
    
    # 사용자 프로필에 기반한 아이템 추천
    user_preferences = user_profile
    recommended_items = []
    for pref in user_preferences:
        neighbors = sorted(G.neighbors(pref), key=lambda x: G[pref][x]['weight'], reverse=True)
        recommended_items.extend(neighbors[:top_n])
    
    # 중복 제거 및 상위 N개 아이템 선택
    recommended_items = list(set(recommended_items))
    recommended_items.sort(key=lambda x: G.degree(x, weight='weight'), reverse=True)
    return recommended_items[:top_n]


In [ ]:
# 순차 패턴 추천
import pandas as pd
from collections import defaultdict
from itertools import combinations

def sequential_pattern_recommender(user_sequences, top_n=5):
    # 사용자 행동 시퀀스 데이터 전처리
    sequences = []
    for user, actions in user_sequences.items():
        sequences.append(actions)
    
    # 순차 패턴 마이닝
    frequent_patterns = defaultdict(int)
    for seq in sequences:
        for i in range(len(seq)):
            for j in range(i+1, len(seq)+1):
                frequent_patterns[tuple(seq[i:j])] += 1
    
    # 순차 패턴 기반 추천
    recommended_items = []
    for pattern, count in sorted(frequent_patterns.items(), key=itemgetter(1), reverse=True):
        for item in pattern:
            if item not in recommended_items:
                recommended_items.append(item)
                if len(recommended_items) >= top_n:
                    break
    
    return recommended_items